In [15]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys, os
sys.path.append('../smc')
sys.path.append('../third_party')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import numpy as np   
import pandas as pd
import scipy.stats as stats
import sys
from tqdm import tqdm

from utils import *     # contains some useful helper functions 
from models import *    # toy models
from solvers import *   # matrix completion solvers
from methods import *
from missingness_estimation import *

In [17]:
r = 8
seed = 1

# Fixed data parameters
max_test_queries = 100            
max_calib_queries = 2000
matrix_generation_seed = 2024    # Data matrix is fixed 

n1 = n2 = 300

model = "RFM"
solver = "svt"
r_solver = 8
prop_obs = 0.3

sd = 0.5

# Other parameters
verbose = True
allow_inf = False
alpha = 0.1

scale=0.2
const=1


k_list = [5]
repetition = 1

In [18]:
#################
# Generate Data #
#################
if model == "RFM":
    mm = RandomFactorizationModel(n1 ,n2, 8)
elif model == "ROM":
    mm = RandomOrthogonalModel(n1 ,n2, 8)
else:
    mm = RandomFactorizationModel(n1 ,n2, 8)

if verbose:
    print('Fixing the ground truth matrix generated from the {} model.\n'.format(model))
    sys.stdout.flush()

_, _, _, M = mm.sample_noisy(sigma=sd, random_state = matrix_generation_seed)

Fixing the ground truth matrix generated from the RFM model.



In [19]:
#####################
# Define Experiment #
#####################
def run_single_experiment(M_true, k, alpha, prop_obs, max_test_queries, max_calib_queries,
                          r, scale, random_state=0):
    res = pd.DataFrame({})


    #--------Observation bias-------#
    #-------------------------------#
    n1, n2 = M_true.shape
    bm = SamplingBias(n1,n2, normalize=False)
    w_obs = bm.block_weights(ratio=alpha, scale=scale, random_state=random_state)

    #-------Generate masks----------#
    #-------------------------------#
    sampler = QuerySampling(n1,n2)
    mask_obs, mask_test = sampler.sample_submask(sub_size=prop_obs, w=w_obs, random_state=random_state)
    n_calib_queries = min(int(0.5 * np.sum(np.sum(mask_obs, axis=1) // k)), max_calib_queries)

    print(f"Estimating missingness with guessed rank {r}...")
    w_obs_est = estimate_P(mask_obs, 1, r=r, const=const)
    print("Done estimating!\n")
    sys.stdout.flush()

    #------Sample test queries------#
    #-------------------------------#
    n_test_queries = min(int(0.99 * np.sum(np.sum(mask_test, axis=1) // k)), max_test_queries)
    _, idxs_test, _ = sampler.sample_train_calib(mask_test, k, calib_size=n_test_queries, random_state=random_state)  
    if verbose:
        print("Training size:{}, calib size: {}, test size: {}\n".format(np.sum(mask_obs)-n_calib_queries*k, n_calib_queries, n_test_queries))
        sys.stdout.flush()


    #------Split train calib--------#
    #-------------------------------#
    mask_train, idxs_calib, _ = sampler.sample_train_calib(mask_obs, k, 
                                calib_size=n_calib_queries, random_state=random_state)

    #--------Model Training---------#
    #-------------------------------#
    print("Running matrix completion algorithm on the splitted training set...")
    sys.stdout.flush()
    if solver == "pmf":
        Mhat, _, _ = pmf_solve(M, mask_train, k=r_solver, verbose=verbose, random_state=random_state)
    elif solver == "svt":
        Mhat = svt_solve(M, mask_train, verbose = verbose, random_state = random_state)
    print("Done training!\n")
    sys.stdout.flush()

    #------Compute intervals--------# 
    #-------------------------------#
    # Evaluate the CI and quantile inflation weights using oracle obs sampling weights
    ci_method = SimulCI(M, Mhat, mask_obs, idxs_calib, k, w_obs=w_obs)
    df = ci_method.get_CI(idxs_test, alpha, allow_inf=allow_inf, store_weights=True)
    lower, upper, is_inf= df.loc[0].lower, df.loc[0].upper, df.loc[0].is_inf
    res = pd.concat([res, evaluate_SCI(lower, upper, k, M, idxs_test, is_inf=is_inf, metric='mean',method="conformal")])
    
    # Evaluate the CI and quantile inflation weights using estimated obs sampling weights
    ci_est = SimulCI(M, Mhat, mask_obs, idxs_calib, k, w_obs=w_obs_est)
    df = ci_est.get_CI(idxs_test, alpha, allow_inf=allow_inf, store_weights=True)
    lower, upper, is_inf= df.loc[0].lower, df.loc[0].upper, df.loc[0].is_inf
    res = pd.concat([res, evaluate_SCI(lower, upper, k, M, idxs_test, is_inf=is_inf, metric='mean',method="est")])

    # Evaluate the estimation gap
    weights_list = ci_method.weights_list
    est_weights_list = ci_est.weights_list
    est_gaps =[0.5*np.sum(np.abs(weights_list[i]-est_weights_list[i])) for i in range(len(weights_list))]
    avg_gap = np.mean(est_gaps)


    res['k'] = k 
    res['avg_gap'] = avg_gap   
    res['Calib_queries'] = n_calib_queries
    res['Train_entries'] = np.sum(mask_train)
    res['Test_queries'] = n_test_queries
    res['random_state'] = random_state
    return res

In [20]:
#####################
#  Run Experiments  #
#####################
results = pd.DataFrame({})

for i in tqdm(range(1, repetition+1), desc="Repetitions", leave=True, position=0):
    #random_state = repetition * (seed-1) + i
    random_state=280
    for k in tqdm(k_list, desc="k", leave=True, position=0):

        res = run_single_experiment(M, k, alpha, prop_obs, max_test_queries, max_calib_queries,
                            r, scale=scale, random_state=random_state)
        
        results = pd.concat([results, res])

k:   0%|                                                                                         | 0/1 [00:00<?, ?it/s]

Estimating missingness with guessed rank 8...
iter: 1
iter: 2
iter: 3
iter: 4
iter: 5
iter: 6
iter: 7
iter: 8
iter: 9
iter: 10
iter: 11
Function value changing by less than progTol
Done estimating!

Training size:17000, calib size: 2000, test size: 100

Running matrix completion algorithm on the splitted training set...
Iteration: 1; Rel error: 1.0000; Rank: 0
Iteration: 11; Rel error: 0.2257; Rank: 8
Iteration: 21; Rel error: 0.1677; Rank: 19
Iteration: 31; Rel error: 0.3503; Rank: 36
Iteration: 41; Rel error: 0.4328; Rank: 50
Iteration: 51; Rel error: 0.4309; Rank: 60
Iteration: 61; Rel error: 0.5889; Rank: 69
Iteration: 71; Rel error: 0.4358; Rank: 46
Iteration: 81; Rel error: 0.6064; Rank: 76
Iteration: 91; Rel error: 0.4526; Rank: 47
Iteration: 101; Rel error: 0.6127; Rank: 79
Iteration: 111; Rel error: 0.4634; Rank: 48
Iteration: 121; Rel error: 0.6147; Rank: 80
Iteration: 131; Rel error: 0.4702; Rank: 49
Iteration: 141; Rel error: 0.6153; Rank: 81
Iteration: 151; Rel error: 0.47

ipdb>  q


Repetitions:   0%|                                                                               | 0/1 [06:41<?, ?it/s]


In [146]:
results

,Query_coverage,Coverage,Size,metric,Inf_prop,Method,k,avg_gap,Calib_queries,Train_entries,Test_queries,random_state
0,0.89,0.97,3.029717,mean,0.0,conformal,5,0.181221,2000,17000,100,280
0,0.89,0.97,3.121167,mean,0.0,est,5,0.181221,2000,17000,100,280
